## TODO

- vectorize `tconfint()`
- vectorize `passToThread()`
  - generate 7300xn1 and 7300xn2 for the matrices of data
  - 7300 x n1 -> 7300 x 1 (`lower`)
  - 7300 x n2 -> 7300 x 1 (`upper`)
  - 7300 x 1 boolean vector, i-th entry is True iff `lower[i] <= delta-true <= upper[i]`
  - ouptut p-value which is proportion of boolean vector that is True

---

In [383]:
import numpy as np
import statsmodels.stats.api as sms
from scipy.stats import t as t
import math
from itertools import combinations
import threading
#from tqdm import tqdm

In [214]:
from src.search import search
from src.bootstrap import bootstrap_ci
from src.perm_test import ttest_ind_vectorized, pval_vectorized

In [215]:
def get_partitions(n1, n2):
    total_length = n1 + n2

    def get_groups(idxs):
        i = 0
        idxs2 = []
        for j in idxs:
            while i < total_length and i < j:
                idxs2.append(i)
                i += 1

            if i == j:
                i += 1

        idxs2 += range(i, total_length)
        return list(idxs) + idxs2

    partitions = np.array([get_groups(idxs) for idxs in combinations(range(total_length), n1)])
    return partitions

In [216]:
def tconfint(alpha, pooled, x1, x2):
    cm = sms.CompareMeans(sms.DescrStatsW(x1), sms.DescrStatsW(x2))
    return cm.tconfint_diff(alpha, usevar="pooled" if pooled else "unequal")

In [217]:
def passToThread(n_samples):
    global n_captured
    
    for _ in range(n_samples):
        x1 = np.random.gamma(gamma1[0], gamma1[1], n1)
        x2 = np.random.gamma(gamma2[0], gamma2[1], n2)

        # TODO vectorize
        t99 = tconfint(0.001, pooled, x1, x2)
        t90 = tconfint(0.20, pooled, x1, x2)

        try:
            lower = search(x1, x2, partitions, t99[0], t90[0])
            upper = search(x1, x2, partitions, t90[1], t99[1])
        except AssertionError:
            continue

        intervals.append((lower, upper))
        n_captured += (lower <= delta_true) * (delta_true <= upper)

In [218]:
alpha = 0.05
alternative = "less"
pooled = True

gamma1 = (2, 5)  # shape k, scale theta
gamma2 = (4, 3)
delta_true = (gamma1[0] * gamma1[1]) - (gamma2[0] * gamma2[1])  # true mean difference

n1, n2 = 12, 8
partitions = get_partitions(n1, n2)

intervals = []
n_captured = 0

In [219]:
%%time
# time how long it takes to find one confidence interval
passToThread(1)

CPU times: user 342 ms, sys: 26.8 ms, total: 369 ms
Wall time: 368 ms


In [220]:
delta_true

-2

In [221]:
n_captured, len(intervals)

(1, 1)

In [364]:
thread_count = 16
threads = [0] * thread_count

remaining = 128
batch_size = remaining // thread_count

intervals = []
n_captured = 0

In [365]:
%%time

for i in range(thread_count):
    n_samples = batch_size if i < thread_count-1 else remaining
    threads[i] = threading.Thread(target=passToThread, args=(n_samples,))
    threads[i].start()
    remaining -= n_samples
    
for thread in threads:
    thread.join()

CPU times: user 1min 9s, sys: 1.42 s, total: 1min 10s
Wall time: 9.6 s


In [224]:
n_captured, len(intervals)

(125, 128)

Multithreading allows us to compute one confidence interval in \~80 ms, even faster than it took to compute the one confidence interval above (\~360 ms).

## Scrap Code

In [369]:
# https://www.statsmodels.org/stable/generated/statsmodels.stats.weightstats.CompareMeans.html
np.random.seed(123)
n1, n2 = 12, 8
n_samples = 1
x1 = np.random.gamma(gamma1[0], gamma1[1], n1)
x2 = np.random.gamma(gamma2[0], gamma2[1], n2)

In [371]:
cm = sms.CompareMeans(sms.DescrStatsW(x1), sms.DescrStatsW(x2))
t99 = cm.tconfint_diff(0.01, usevar="pooled" if pooled else "unequal")
t90 = cm.tconfint_diff(0.1, usevar="pooled" if pooled else "unequal") 
print(t90)
print(t99)

(-7.148602688245627, 3.544779427557045)
(-10.677094684430186, 7.073271423741604)


In [372]:
lower = search(x1, x2, partitions, t99[0], t90[0])
upper = search(x1, x2, partitions, t90[1], t99[1])
lower, upper, delta_true

(-10.670203098500139, 4.867963926126254, -2)

In [373]:
np.random.seed(123)
n1, n2 = 12, 8
n_samples = 1
x1 = np.random.gamma(gamma1[0], gamma1[1], (n_samples, n1))
x2 = np.random.gamma(gamma2[0], gamma2[1], (n_samples, n2))

In [374]:
# TODO alternative="unequal", etc.
# TODO this code is almost identical to ttest_ind_vectorized
#      merge into same function with parameter to switch return value
def tconfint_vectorized(x1s, x2s, alpha=0.05, pooled=True, alternative="unequal"):
    n1, n2 = x1s.shape[-1], x2s.shape[-1]
    #print("n1 =", n1, "n2 =", n2)
    sum1 = np.sum(x1s, axis=-1)
    sum2 = np.sum(x2s, axis=-1)
    #print("sums", sum1, sum2)

    mean1 = sum1 / n1
    mean2 = sum2 / n2
    #print("means", mean1, mean2)

    sample_var = lambda x, mean, n: (np.sum(x**2, axis=-1) - n*mean**2) / (n-1)
    var1 = sample_var(x1s, mean1, n1)
    var2 = sample_var(x2s, mean2, n2)
    #print("sample variances", var1, var2)

    pooled_var = ((n1-1)*var1 + (n2-1)*var2) / (n1+n2-2)
    #print("pooled var", pooled_var)
    denom = np.sqrt(pooled_var * (1/n1 + 1/n2))

    t_crit = t.ppf(q=1-alpha/2, df=n1+n2-2)  # scipy.stats.t.ppf()
    lower = (mean1 - mean2) - t_crit*denom
    upper = (mean1 - mean2) + t_crit*denom
    return lower, upper

In [375]:
t90 = tconfint_vectorized(x1, x2, alpha=0.1)
t99 = tconfint_vectorized(x1, x2, alpha=0.01)
print(t90)
print(t99)

(array([-7.14860269]), array([3.54477943]))
(array([-10.67709468]), array([7.07327142]))


In [376]:
i = 0  # sample
lower = search(x1[i], x2[i], partitions, t99[0][i], t90[0][i])
upper = search(x1[i], x2[i], partitions, t90[1][i], t99[1][i])

In [377]:
lower, upper

(-10.670203098500139, 4.867963926126254)

In [378]:
# TODO this is only partially vectorized; still need to vectorize `search`
def passToThread_vectorized(n_samples):
    global n_captured
    np.random.seed(123)

    x1 = np.random.gamma(gamma1[0], gamma1[1], (n_samples, n1))
    x2 = np.random.gamma(gamma2[0], gamma2[1], (n_samples, n2))
    
    t99 = tconfint_vectorized(x1, x2, alpha=0.01)
    t90 = tconfint_vectorized(x1, x2, alpha=0.1)
    
    # this needs to be vectorized
    for i in range(n_samples):
        try:
            lower = search(x1[i], x2[i], partitions, t99[0][i], t90[0][i])
            upper = search(x1[i], x2[i], partitions, t90[1][i], t99[1][i])
        except AssertionError:
            #print("AssertionError: Alpha not contained in estimated bounds")
            continue
        #print(i, ": (", lower, ",", upper, ")")
        n_captured += (lower <= delta_true) * (delta_true <= upper)

In [379]:
%%time
n_captured = 0
passToThread_vectorized(32)
n_captured

CPU times: user 9.5 s, sys: 31.8 ms, total: 9.53 s
Wall time: 9.53 s


32

In [380]:
%%time
n_captured = 0
passToThread(32)
n_captured

CPU times: user 10.4 s, sys: 72.5 ms, total: 10.5 s
Wall time: 10.5 s


30

In [381]:
thread_count = 16
threads = [0] * thread_count

remaining = 128
batch_size = remaining // thread_count

intervals = []
n_captured = 0

In [382]:
%%time

for i in range(thread_count):
    n_samples = batch_size if i < thread_count-1 else remaining
    threads[i] = threading.Thread(target=passToThread_vectorized, args=(n_samples,))
    threads[i].start()
    remaining -= n_samples
    
for thread in threads:
    thread.join()

CPU times: user 1min 2s, sys: 1.56 s, total: 1min 4s
Wall time: 8.79 s
